In [1]:
import torch
import torchaudio
import torch.utils.data as data
import torchvision.transforms as transforms
import numpy as np
from pytorchnsynthmaster.pytorch_nsynth.nsynth import NSynth, SignalTransformation
import sys
torch.cuda.is_available()

/home/alexis/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
toFloat = transforms.Lambda(lambda x: x / np.iinfo(np.int16).max)

In [3]:
dataset = NSynth(
        "nsynth-valid",
        transform=toFloat,
        blacklist_pattern=["string"],  # blacklist string instrument
        categorical_field_list=["instrument_family", "instrument_source"])

In [4]:
loader = data.DataLoader(dataset, batch_size=32, shuffle=True)

# Coger el archivo de audio con source 0 y clasificar por familias


### Families
0 - Bass

1 - Brass

2 - Flute

3 - Guitar

4 - Keyboard

5 - Mallet

6 - Organ

7 - Reed

8 - String

9 - Synth_lead

10 - Vocal

In [5]:
# print(dataset.__getitem__(0))
# print(dataset.__getitem__(0)[3]["note_str"])
# print(dataset.__getitem__(0)[3]["instrument_source"])
print(dataset.getAudioCharacteristics(0))

{'fichero': 'bass_synthetic_033-087-050.wav', 'familia': 2}


In [8]:
audioCharacteristics=dataset.getAudioCharacteristics(0)
signalTransformation = SignalTransformation(fichero=audioCharacteristics["fichero"],label=audioCharacteristics["familia"])
print(signalTransformation.generarSpectrograma())
# print(signalTransformation.generarSpectrograma())


tensor([[[2.7021e+03, 1.6292e+04, 2.5236e+04,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [6.5048e+02, 2.0570e+01, 1.7952e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [5.1166e+01, 9.8825e-01, 4.6492e-02,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [1.1064e-05, 2.6957e-06, 1.2861e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [5.8676e-06, 2.8448e-06, 1.3947e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [2.0117e-06, 3.2150e-06, 3.0517e-06,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]]])


In [ ]:
# SAMPLE_RATE=16000
# mel_spectrogram = torchaudio.transforms.MelSpectrogram(sample_rate=SAMPLE_RATE,
#                                                       n_fft=1024, 
#                                                       hop_length=512,
#                                                       n_mels=64)